In [39]:
# 드라이브 마운트하기
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#force_remount=True 는 기존 마운트와 상관없이 다시 마운트하라는 뜻

Mounted at /content/drive


In [40]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import GRU, Dense, Dropout

In [41]:
data = pd.read_csv('/content/drive/MyDrive/농산물가격예측/public_data/train.csv')
data.head(5)

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,5475.0,92334.0,1704.0,6359.0,1331.0,40028.0,348.0,4374.9,13242.0,16550.5,2339.0,10528.0,1729.0,13885.0,804.0,3853.0,3703.0,15797.0,2576.0,14634.0,1474.0,30950.0,1621.0,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2,2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,141638.0,5210.0,994328.1,1716.0,262615.5,1212.0,1074699.1,345.0,122613.5,9923.0,427435.1,2153.0,82113.5,3960.0,558950.2,794.0,104930.3,4871.0,277326.5,2440.0,159800.0,1750.0,291057.0,1834.0,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
4,2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,126207.8,5387.0,787716.0,1715.0,221850.5,1197.0,825681.9,350.0,79055.9,9529.0,334636.8,2220.0,80144.0,3333.0,444353.7,763.0,100699.5,5129.0,218465.2,2437.0,153084.0,1822.0,194626.5,1833.0,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0


In [42]:
# 시간 데이터 인덱스
data['date'] = pd.to_datetime(data['date'])
data = data.set_index(data['date'])
data = data.iloc[:, 1:]
data.head(5)

,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,5475.0,92334.0,1704.0,6359.0,1331.0,40028.0,348.0,4374.9,13242.0,16550.5,2339.0,10528.0,1729.0,13885.0,804.0,3853.0,3703.0,15797.0,2576.0,14634.0,1474.0,30950.0,1621.0,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,141638.0,5210.0,994328.1,1716.0,262615.5,1212.0,1074699.1,345.0,122613.5,9923.0,427435.1,2153.0,82113.5,3960.0,558950.2,794.0,104930.3,4871.0,277326.5,2440.0,159800.0,1750.0,291057.0,1834.0,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,126207.8,5387.0,787716.0,1715.0,221850.5,1197.0,825681.9,350.0,79055.9,9529.0,334636.8,2220.0,80144.0,3333.0,444353.7,763.0,100699.5,5129.0,218465.2,2437.0,153084.0,1822.0,194626.5,1833.0,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0


In [43]:
# 결측치 보간
for i, row in data.iterrows():
    for col in data.columns:
        if row[col] == 0:
            data.loc[i, col] = np.nan

data = data.apply(lambda x: x.interpolate(), axis=0)
data.head()

,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01,금요일,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-02,토요일,80860.00,329.0,80272.00,360.0,122787.50,1281.0,3.0,11000.0,15019.0,5475.0,92334.00,1704.0,6359.00,1331.0,40028.00,348.0,4374.9,13242.0,16550.5,2339.0,10528.00,1729.0,13885.0,804.0,3853.00,3703.0,15797.00,2576.0,14634.0,1474.0,30950.0,1621.0,5125.00,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,NaN,NaN
2016-01-03,일요일,751801.25,403.5,889962.85,371.0,1218933.25,1258.0,351.0,7732.0,78328.5,5342.5,543331.05,1710.0,134487.25,1271.5,557363.55,346.5,63494.2,11582.5,221992.8,2246.0,46320.75,2844.5,286417.6,799.0,54391.65,4287.0,146561.75,2508.0,87217.0,1612.0,161003.5,1727.5,21825.25,8433.0,250568.0,2077.5,319849.0,2216.0,1791.9,2949.5,NaN,NaN
2016-01-04,월요일,1422742.50,478.0,1699653.70,382.0,2315079.00,1235.0,699.0,4464.0,141638.0,5210.0,994328.10,1716.0,262615.50,1212.0,1074699.10,345.0,122613.5,9923.0,427435.1,2153.0,82113.50,3960.0,558950.2,794.0,104930.30,4871.0,277326.50,2440.0,159800.0,1750.0,291057.0,1834.0,38525.50,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,NaN,NaN
2016-01-05,화요일,1167241.00,442.0,1423482.30,422.0,2092960.10,1213.0,1112.6,4342.0,126207.8,5387.0,787716.00,1715.0,221850.50,1197.0,825681.90,350.0,79055.9,9529.0,334636.8,2220.0,80144.00,3333.0,444353.7,763.0,100699.50,5129.0,218465.20,2437.0,153084.0,1822.0,194626.5,1833.0,32615.00,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,NaN,NaN


In [44]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
배추_거래량(kg),1732.0,1.284112e+06,763835.762576,28.0,8.844770e+05,1113519.200,1.434405e+06,5210215.0
배추_가격(원/kg),1732.0,7.034535e+02,360.538080,243.0,4.537500e+02,662.000,8.386250e+02,5000.0
무_거래량(kg),1732.0,1.472973e+06,511004.155751,2396.0,1.199686e+06,1428196.300,1.695836e+06,3859745.9
무_가격(원/kg),1732.0,5.740413e+02,236.670430,209.0,3.710000e+02,509.500,7.480000e+02,1360.0
양파_거래량(kg),1732.0,1.921190e+06,733543.374633,500.0,1.517797e+06,1787886.125,2.244573e+06,5024753.4
양파_가격(원/kg),1732.0,7.665716e+02,267.680667,287.0,5.420000e+02,740.500,9.710000e+02,2000.0
건고추_거래량(kg),1732.0,3.790856e+03,24132.908376,1.0,2.723000e+02,739.450,2.061800e+03,413656.0
건고추_가격(원/kg),1732.0,1.143162e+04,8214.516373,21.0,7.144500e+03,10142.250,1.329675e+04,170836.0
마늘_거래량(kg),1732.0,1.589288e+05,139926.815733,90.0,8.720560e+04,112786.175,1.627027e+05,1092179.7
마늘_가격(원/kg),1732.0,4.054978e+03,1051.067778,1500.0,2.997875e+03,4159.500,4.899250e+03,6415.0


In [45]:
# 평가식
def nmae(y_true, y_pred):
    score = np.mean(np.abs(y_true - y_pred) / y_true)
    return score

In [46]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

def normalize(data, col, scaler):
    norm_data = data.iloc[1:, col:col + 2]
    for col in norm_data:
        norm_data[col] = scaler.fit_transform(np.array(norm_data[col]).reshape(-1, 1))
    
    return norm_data

In [47]:
def load_data(norm_data, window_size=28, future_size=28, train_size=0.9):    
    X = []
    y = []

    for i in range(len(norm_data) - window_size - future_size):
        X.append(norm_data.iloc[i:i + window_size])
        y.append(norm_data.iloc[i + window_size:i + window_size + future_size, 1])

    X = np.array(X)
    y = np.array(y)

    train_idx = round(len(X) * train_size)
    train_X = X[:train_idx]
    train_y = y[:train_idx]
    valid_X = X[train_idx:]
    valid_y = y[train_idx:]

    return train_X, train_y, valid_X, valid_y

In [48]:
def build_model():
    model = Sequential()
    model.add(GRU(50, return_sequences=True, input_shape=(train_X.shape[1],2)))
    model.add(Dropout(0.2))
    model.add(GRU(100, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(28, activation = "linear"))

    model.compile(loss='mse', optimizer='adam')
    return model

In [50]:
score = []
for col in list(range(1, 42, 2)):
    norm_data = normalize(data, col, scaler)
    train_X, train_y, test_X, test_y = load_data(norm_data)
    model = build_model()
    model.fit(train_X, train_y, batch_size=10, epochs=3, validation_data=(valid_X, valid_y), verbose=0)

    y_pred = model.predict(test_X)
    y_pred = scaler.inverse_transform(y_pred)
    y_true = scaler.inverse_transform(test_y)

    print(f'{data.columns[col + 1]} 모델 NMAE:', nmae(y_true, y_pred))
    score.append(nmae(y_true, y_pred))

배추_가격(원/kg) 모델 NMAE: 0.1818911127619003
무_가격(원/kg) 모델 NMAE: 0.15867410498551834
양파_가격(원/kg) 모델 NMAE: 0.11433315040631828
건고추_가격(원/kg) 모델 NMAE: 0.36356654194606053
마늘_가격(원/kg) 모델 NMAE: 0.1465647442248108
대파_가격(원/kg) 모델 NMAE: 0.16706647900658111
얼갈이배추_가격(원/kg) 모델 NMAE: 0.18269716437454891
양배추_가격(원/kg) 모델 NMAE: 0.3228459633615227
깻잎_가격(원/kg) 모델 NMAE: 0.19945359839147014
시금치_가격(원/kg) 모델 NMAE: 0.23903237508406758
미나리_가격(원/kg) 모델 NMAE: 0.363436807672403
당근_가격(원/kg) 모델 NMAE: 0.09310620551346867
파프리카_가격(원/kg) 모델 NMAE: 0.22755799336993196
새송이_가격(원/kg) 모델 NMAE: 0.09429657722582717
팽이버섯_가격(원/kg) 모델 NMAE: 0.15940842871591174
토마토_가격(원/kg) 모델 NMAE: 0.23646144128364577
청상추_가격(원/kg) 모델 NMAE: 0.3824169301031226
백다다기_가격(원/kg) 모델 NMAE: 0.23631832939838152
애호박_가격(원/kg) 모델 NMAE: 0.2572080489069946
캠벨얼리_가격(원/kg) 모델 NMAE: 0.17856339554657397
샤인마스캇_가격(원/kg) 모델 NMAE: nan


In [53]:
np.mean(score[:-1])

0.21524496961395298